### Graphing Module

In [2]:
# execute `brew install graphviz` in terminal to install directly to path

from graphviz import Digraph

def trace(root):
  # builds a set of all nodes and edges in a graph
  nodes, edges = set(), set()
  def build(v):
    if v not in nodes:
      nodes.add(v)
      for child in v._prev:
        edges.add((child, v))
        build(child)
  build(root)
  return nodes, edges

def draw_dot(root):
  dot = Digraph(format='svg', graph_attr={'rankdir': 'LR'}) # LR = left to right
  
  nodes, edges = trace(root)
  for n in nodes:
    uid = str(id(n))
    # for any value in the graph, create a rectangular ('record') node for it
    dot.node(name = uid, label = "{ %s | data %.4f | grad %.4f }" % (n.label, n.data, n.grad), shape='record')
    if n._op:
      # if this value is a result of some operation, create an op node for it
      dot.node(name = uid + n._op, label = n._op)
      # and connect this node to it
      dot.edge(uid + n._op, uid)

  for n1, n2 in edges:
    # connect n1 to the op node of n2
    dot.edge(str(id(n1)), str(id(n2)) + n2._op)

  return dot

### Main Module

In [3]:
import math

class Value:

        
        self.data = data
        self.grad = 0.0
        self._op = _op
        self._prev = set(_children)
        self._backward = lambda: None # by default, doesn't do anything
        self.label = label

    def _backward(self):
        self._backward = _backward

    def __repr__(self):
        return f"Value(data={self.data})"

    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        # this line ensures that "other" is a Value object if it is a number
        
        out = Value( self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += 1.0 * out.grad # make sure this is cummulative, else can't use the same variable
            other.grad += 1.0 * out.grad # make sure this is cummulative, else can't use the same variable
        out._backward = _backward

        return out
    
    def __radd__(self, other):
        return self + other

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)

        out = Value( self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad # make sure this is cummulative, else can't use the same variable
            other.grad += self.data * out.grad # make sure this is cummulative, else can't use the same variable
        out._backward = _backward
        
        return out

    def __rmul__(self,other):
        """
        This handles instances where you multiply 2 * x which gives an error, as it
        results in 2.__mul__(x) but 2 is not a Value object
        """
        return self * other

    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supporting integer / float powers"
        out = Value( self.data ** other , (self, ), f"pwr of {other}")

        def _backward():
            self.grad += other * (self.data ** (other-1)) * out.grad
        out._backward = _backward

        return out

    def __neg__(self):
        return self * -1

    def __sub__(self, other):
        return self + (-other)
    
    def __truediv__(self, other):
        return self * (other ** -1)

    def tanh(self):
        x = self.data
        t = (math.exp( 2*x ) - 1 ) / (math.exp( 2*x ) + 1)
        out = Value( t , (self, ), "tanh")
        
        def _backward():
            self.grad += ( 1 - t**2 ) * out.grad # make sure this is cummulative, else can't use the same variable
        out._backward = _backward
        
        return out

    def exp(self):
        out = Value( math.exp(self.data) , (self, ), "exp")

        def _backward():
            self.grad += out.data * out.grad
        out._backward = _backward

        return out

    def backward(self):
        # topological sort algorithm
        topo = []
        visited = set()
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)
        # building the topology of self
        build_topo(self)
        # initialise the gradient of the end
        self.grad = 1.0
        # calling _backward function for each node starting from the end
        for i in reversed(topo):
            i._backward()

In [ ]:
x1 = Value(2.0, label ="x1")
x2 = Value(0.0, label ="x2")

w1 = Value(-3.0, label ="w1")
w2 = Value(1.0, label ="w2")

# bias of neuron
b = Value(6.88137, label = "b")

# x1w1 + x2w2 + b
x1w1 = x1*w1 ; x1w1.label = "x1w1"
x2w2 = x2*w2 ; x2w2.label = "x2w2"
x1w1x2w2 = x1w1 + x2w2 ; x1w1x2w2.label = "x1w1 + x2w2"

n = x1w1x2w2 + b; n.label = "x1w1 + x2w2 + b"

o = n.tanh(); o.label = "tanh( x1w1 + x2w2 + b )"

In [ ]:
o.backward()
draw_dot(o)

In [ ]:
# Creating a new line to see if the backprop works for our exp

x1 = Value(2.0, label ="x1")
x2 = Value(0.0, label ="x2")

w1 = Value(-3.0, label ="w1")
w2 = Value(1.0, label ="w2")

# bias of neuron
b = Value(6.88137, label = "b")

# x1w1 + x2w2 + b
x1w1 = x1*w1 ; x1w1.label = "x1w1"
x2w2 = x2*w2 ; x2w2.label = "x2w2"
x1w1x2w2 = x1w1 + x2w2 ; x1w1x2w2.label = "x1w1 + x2w2"

n = x1w1x2w2 + b; n.label = "n"

# ---
e = (n * 2 ).exp()
o = (e - 1) / (e + 1)
# ---
o.label = "o"
o.backward()
draw_dot(o)

In [ ]:
a = Value(2.0)
b = Value(4.0)
a/b